In [71]:
import numpy as np
from sparse_vector import *
from vector_polynomial_isomorphism import *
from minor_flattening import *

We note that we are using the base field `QQ` for all computations currently

In [72]:
# Let us initialize constants and vector spaces.

N = 2*2*3
n = [2,2,3]
indeterminates = form_indeterminates(n)
parameters = [var("".join(["p_", str(i+1)])) for i in range(14)]
R = PolynomialRing(QQ, indeterminates)
R_parameterized = PolynomialRing(QQ, indeterminates + parameters)
R.inject_variables()
R_parameterized.inject_variables()

glN = MatrixSpace(QQ, N)
dimV3 = binomial(N + 3 - 1, 3)
V3 = VectorSpace(QQ, dimV3)
dimV4 = binomial(N + 4 - 1, 4)
V4 = VectorSpace(QQ, dimV4)

vp_iso3 = vector_polynomial_isomorphism(R, V3, indeterminates, 3)
vp_iso4 = vector_polynomial_isomorphism(R, V4, indeterminates, 4)

Defining x_000, x_001, x_002, x_010, x_011, x_012, x_100, x_101, x_102, x_110, x_111, x_112
Defining x_000, x_001, x_002, x_010, x_011, x_012, x_100, x_101, x_102, x_110, x_111, x_112, p_1, p_2, p_3, p_4, p_5, p_6, p_7, p_8, p_9, p_10, p_11, p_12, p_13, p_14


In [73]:
# deg 3 generators
f_deg3 = get_flattening_minors(R, n, 3)
f_vec3 = [vp_iso3.polynomial_to_vector(fi) for fi in f_deg3]

# deg 4 generators

# first add the 48 generators created from multiplying single variables by the flattening 3 minors
f_deg4 = []
for vari in indeterminates:
    for fi in f_deg3:
        f_deg4.append(vari*fi)

# L is the parameterization of the tangential variety
L = []
a = [[p_1,p_2], [p_8, p_9]]
b = [[p_3,p_4], [p_10, p_11]]
c = [[p_5,p_6,p_7], [p_12,p_13,p_14]]
for i in range(2):
    for j in range(2):
        for k in range(3):
            L.append(R_parameterized(var("".join(["x_", str(i), str(j), str(k)])) - a[1][i]*b[0][j]*c[0][k] - a[0][i]*b[1][j]*c[0][k] - a[0][i]*b[0][j]*c[1][k]))

# elimination ideal, the ideal of the tangential variety
J = Ideal(L).elimination_ideal([p_1, p_2, p_3, p_4, p_5, p_6, p_7, p_8, p_9, p_10, p_11, p_12, p_13, p_14])
f_deg4 += J.gens()[4:] # the first 4 out of 10 are degree 3, just ignore those in this case
f_vec4 = [vp_iso4.polynomial_to_vector(R(fi)) for fi in f_deg4]


In [74]:
W3 = V3.subspace(f_vec3)
print(W3)

W4 = V4.subspace(f_vec4)
print(W4)

Vector space of degree 364 and dimension 4 over Rational Field
Basis matrix:
4 x 364 dense matrix over Rational Field
Vector space of degree 1365 and dimension 51 over Rational Field
Basis matrix:
51 x 1365 dense matrix over Rational Field


In [75]:
W_perp3 = W3.complement()
print(W_perp3)
p3 = W3.dimension()
q3 = W_perp3.dimension()

W_perp4 = W4.complement()
print(W_perp4)
p4 = W4.dimension()
q4 = W_perp4.dimension()

Vector space of degree 364 and dimension 360 over Rational Field
Basis matrix:
360 x 364 dense matrix over Rational Field
Vector space of degree 1365 and dimension 1314 over Rational Field
Basis matrix:
1314 x 1365 dense matrix over Rational Field


In [76]:
def E_action(a, b, grad_fl):
    sum = -1*indeterminates[b]*grad_fl[a]
    return R(sum)

We note `M_f` $:= \tilde{M}_f$

In [77]:
g_sparse3 = [sparsify(W_perp3.gen(k)) for k in range(q3)]
g_sparse4 = [sparsify(W_perp4.gen(k)) for k in range(q4)]

In [78]:
E_action_fl = [[0 for j in range(N)] for i in range(N)]

# Construct M_f3
M_f3 = zero_matrix(QQ, p3*q3, N^2, sparse=True)
for l in range(p3):
    grad_fl = vp_iso3.vector_to_polynomial(W3.gen(l)).gradient()
    for i in range(N):
        for j in range(N):
            E_action_fl[i][j] = sparsify(vp_iso3.polynomial_to_vector(E_action(i,j,grad_fl)))
    for k in range(q3):
        for i in range(N):
            for j in range(N):
                M_f3[(q3*l) + k,(N*i) + j] = sparse_inner_product(E_action_fl[i][j], g_sparse3[k])
                
# Construct M_f4
M_f4 = zero_matrix(QQ, p4*q4, N^2, sparse=True)
for l in range(p4):
    grad_fl = vp_iso4.vector_to_polynomial(W4.gen(l)).gradient()
    for i in range(N):
        for j in range(N):
            E_action_fl[i][j] = sparsify(vp_iso4.polynomial_to_vector(E_action(i,j,grad_fl)))
    for k in range(q4):
        for i in range(N):
            for j in range(N):
                M_f4[(q4*l) + k,(N*i) + j] = sparse_inner_product(E_action_fl[i][j], g_sparse4[k])

In [81]:
ns3 = M_f3.right_kernel()
print('dim(ker(M_f3)) =', ns3.dimension())

ns4 = M_f4.right_kernel()
print('dim(ker(M_f4)) =', ns4.dimension())

print('dimension of [ker(M_f3) intersected with ker(M_f4)] = ', ns4.intersection(ns3).dimension())

dim(ker(M_f3)) = 24
dim(ker(M_f4)) = 15
dimension of [ker(M_f3) intersected with ker(M_f4)] =  15


In [82]:
print('expected:', sum([ni^2-1 for ni in n]) + 1)

expected: 15
